# SoundMime.com

Imitates the voice of a human voice.

- Soundstream for encoder/decoder.
- SoundStorm for text-to-speech generation

In [3]:
print("hello")

hello


In [5]:
import torch
import torchaudio
import os
curr_dir = os.getcwd()

In [6]:
from audiolm_pytorch import EncodecWrapper
encodec = EncodecWrapper()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\jung2\OneDrive\G Drive\My Projects\Coding Projects\soundstorm-pytorch\.venv\Lib\site-packages\IPython\core\interactiveshell.py", line 3577, in run_code
  File "C:\Windows\Temp\ipykernel_18492\377550591.py", line 1, in <module>
    from audiolm_pytorch import EncodecWrapper
ModuleNotFoundError: No module named 'audiolm_pytorch'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\jung2\OneDrive\G Drive\My Projects\Coding Projects\soundstorm-pytorch\.venv\Lib\site-packages\pygments\styles\__init__.py", line 45, in get_style_by_name
ModuleNotFoundError: No module named 'pygments.styles.default'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\jung2\OneDrive\G Drive\My Projects\Coding Projects\soundstorm-pytorch\.venv\Lib\site-packages\IPython\core\interactiveshell.py", line 2168, in showtraceback
 

In [3]:
import soundstorm_pytorch
# torchaudio.set_audio_backend("soundfile") 
print(torchaudio.list_audio_backends())

TypeError: cannot unpack non-iterable NoneType object

# Choose your pretrained models

In [ ]:
import torchaudio
from soundstream import from_pretrained, load

# Get audio file path
curr_dir = os.getcwd()
file_path = os.path.join(curr_dir, 'sample1.wav')

# Load the waveform
waveform = load(file_path)
audio_codec = from_pretrained()  # downloads model from Hugging Face
print(waveform.shape)

# quantized = audio_codec(waveform, mode='encode')
# recovered = audio_codec(quantized, mode='decode')

# torchaudio.save('out.wav', recovered[0], 16000)

In [ ]:
print(type(audio_codec))

In [ ]:
from soundstream import load

# Provide initial audio input
audio = load(file_path)
audio = audio.squeeze(0)
print(f"audio.shape: {audio.shape}")

# encoded_features = model.encoder(audio)  # Ensure audio is in the correct format
# # Pass the features through the quantizer to get the codes
# quantized, codes, _ = model.quantizer(encoded_features.permute(0, 2, 1))  # Adjust dimensions if needed

# # quantized = audio_codec(waveform, mode='encode')
# # recovered = audio_codec(quantized, mode='decode')

# # torchaudio.save('out.wav', recovered[0], 16000)


# # # Identify soundstream to utilize.
# # encodec = EncodecWrapper()

# # # Use encodec as soundstream
# # soundstream = encodec 
# # # or train your own soundstream

# # Save the codes to a file
# torch.save(codes, "codes.pt")


In [ ]:
waveform, sample_rate = torchaudio.load(file_path)
print(file_path)
print(waveform.size())
print(sample_rate)

# AudioLM
## SoundStream & Encodec
Neural audio compression. It focuses on compressing audio data efficiently while preserving as much quality as possible at low bitrates, aimed at real-time audio streaming or storage.

### Train the Soundstream

In [ ]:
from audiolm_pytorch import SoundStream, SoundStreamTrainer

soundstream = SoundStream(
    codebook_size = 4096,
    rq_num_quantizers = 8,
    rq_groups = 2,                       # this paper proposes using multi-headed residual vector quantization - https://arxiv.org/abs/2305.02765
    use_lookup_free_quantizer = True,    # whether to use residual lookup free quantization - there are now reports of successful usage of this unpublished technique
    use_finite_scalar_quantizer = False, # whether to use residual finite scalar quantization
    attn_window_size = 128,              # local attention receptive field at bottleneck
    attn_depth = 2                       # 2 local attention transformer blocks - the soundstream folks were not experts with attention, so i took the liberty to add some. encodec went with lstms, but attention should be better
)

folder_path = str(os.path.join(curr_dir,'audio_samples'))

trainer = SoundStreamTrainer(
    soundstream,
    folder = folder_path,
    batch_size = 3,
    grad_accum_every = 8,         # effective batch size of 24
    data_max_length_seconds = 2,  # train on 2 second audio
    num_train_steps = 1_000_000
).cpu()

trainer.train()

# after a lot of training, you can test the autoencoding as so
soundstream.eval() # your soundstream must be in eval mode, to avoid having the residual dropout of the residual VQ necessary for training

# Provide audio input
# audio = torch.randn(10080).cpu() 

# Reconstruct audio using soundstream
recons = soundstream(audio, return_recons_only = True) # (1, 10080) - 1 channel # reconstructed audio signal

### Trained Soundstream as Tokenizer

In [ ]:
from audiolm_pytorch import SoundStream, SoundStreamTrainer

audio = audio
torch.load(soundstream, "results\soundstream0.pt")
# codes = soundstream.tokenize(audio)

# # you can now train anything with the codebook ids

# recon_audio_from_codes = soundstream.decode_from_codebook_indices(codes)



# # sanity check

# assert torch.allclose(
#     recon_audio_from_codes,
#     soundstream(audio, return_recons_only = True)
# )

# SoundStorm

In [ ]:
import torch
from soundstorm_pytorch import SoundStorm, ConformerWrapper

conformer = ConformerWrapper(
    codebook_size = 1024,
    num_quantizers = 16,
    conformer = dict(
        dim = 512,
        depth = 2
    ),
)

model = SoundStorm(
    conformer,
    steps = 18,          # 18 steps, as in original maskgit paper
    schedule = 'cosine'  # currently the best schedule is cosine
)

# get your pre-encoded codebook ids from the soundstream from a lot of raw audio
codes = torch.load('codes.pt') # (batch, seq, num residual VQ)

# do the below in a loop for a ton of data
loss, _ = model(codes)
loss.backward()

# model can now generate in 18 steps. ~2 seconds sounds reasonable

generated = model.generate(1024, batch_size = 2) # (2, 1024)

In [ ]:
import torch
from soundstorm_pytorch import SoundStorm, ConformerWrapper, Conformer, SoundStream

conformer = ConformerWrapper(
    codebook_size = 1024,
    num_quantizers = 12,
    conformer = dict(
        dim = 512,
        depth = 2
    ),
)

soundstream = SoundStream(
    codebook_size = 1024,
    rq_num_quantizers = 12,
    attn_window_size = 128,
    attn_depth = 2
)

model = SoundStorm(
    conformer,
    soundstream = soundstream   # pass in the soundstream
)

# find as much audio you'd like the model to learn

# audio = torch.randn(2, 10080)

# course it through the model and take a gazillion tiny steps

loss, _ = model(audio)
loss.backward()

# and now you can generate state-of-the-art speech

generated_audio = model.generate(seconds = 30, batch_size = 2)  # generate 30 seconds of audio (it will calculate the length in seconds based off the sampling frequency and cumulative downsamples in the soundstream passed in above)

In [ ]:
print(generated_audio.shape)
# Ensure the generated audio is in the correct shape
# (this assumes the model generates in [batch_size, num_channels, num_frames] format)
# If the audio is stereo, ensure the shape is [2, num_frames]
# Save to .wav file using torchaudio
torchaudio.save('generated_audio.wav', generated_audio, 16000)  # 16000 Hz sample rate as an example

print("Audio generated and saved as 'generated_audio.wav'")

In [ ]:
from spear_tts_pytorch import TextToSemantic
from sentence_transformers import SentenceTransformer
text_to_semantic = SentenceTransformer('all-MiniLM-L6-v2')

text_to_semantic = TextToSemantic(
    dim = 512,
    source_depth = 12,
    target_depth = 12,
    num_text_token_ids = 50000,
    num_semantic_token_ids = 20000,
    use_openai_tokenizer = False
)

# # load the trained text-to-semantic transformer

# text_to_semantic.load(model)

# pass it into the soundstorm

model_tts = SoundStorm(
    conformer,
    soundstream = ss_model,
    spear_tts_text_to_semantic = text_to_semantic
).cpu()

# and now you can generate state-of-the-art speech

generated_speech = model_tts.generate(
    texts = [
        'the rain in spain stays mainly in the plain',
        'the quick brown fox jumps over the lazy dog'
    ],
    seconds=30,  # specify the number of seconds of audio to generate
    batch_size=2
) # (2, n) - raw waveform decoded from soundstream